# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#loading the result csv from weatherpy.
cities_df = pd.read_csv('../Output/cities_result.csv', index_col= False)
cities_df = cities_df.drop('Unnamed: 0',axis = 1)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,bosaso,0.0,SO,1.583268e+09,77.0,11.28,49.18,76.87,9.48
1,riohacha,81.0,CO,1.583268e+09,62.0,11.54,-72.91,86.00,12.75
2,namatanai,100.0,PG,1.583268e+09,72.0,-3.67,152.43,84.83,4.81
3,kavieng,100.0,PG,1.583268e+09,71.0,-2.57,150.80,84.79,3.47
4,victoria,40.0,HK,1.583268e+09,73.0,22.29,114.16,70.00,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
figure_layout = {
    'width': '1000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(center=(0, 0), zoom_level=2)
locations = cities_df[["Lat", "Lng"]]

# 
humidity = cities_df["Humidity"]

# Plot Heatmap

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True,
                                point_radius=15,opacity=0.5)


# Add layer
fig.add_layer(heat_layer)

# Display figure


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
#stripping the white space from my column name
cities_df.columns = cities_df.columns.str.strip()

#narrowing the cities to fit required conditions
cities_df_temp =cities_df[(cities_df['Max_Temp'] > 70) & (cities_df['Max_Temp'] < 80)]
cities_df_wind = cities_df_temp[cities_df_temp['Wind_Speed'] < 10]
hotel_df= cities_df_wind[cities_df_wind['Cloudiness'] == 0]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,bosaso,0.0,SO,1.583268e+09,77.0,11.28,49.18,76.87,9.48
53,akyab,0.0,MM,1.583268e+09,86.0,20.15,92.90,71.69,8.84
62,hermanus,0.0,ZA,1.583268e+09,68.0,-34.42,19.23,75.00,9.95
63,araouane,0.0,ML,1.583268e+09,5.0,18.90,-3.53,79.45,8.52
76,kidal,0.0,ML,1.583268e+09,7.0,18.44,1.41,77.36,8.32
79,cape town,0.0,ZA,1.583268e+09,73.0,-33.93,18.42,75.00,1.12
89,bredasdorp,0.0,ZA,1.583268e+09,68.0,-34.53,20.04,73.00,2.64
101,tessalit,0.0,ML,1.583268e+09,6.0,20.20,1.01,76.75,8.95
152,carnarvon,0.0,AU,1.583268e+09,100.0,-24.87,113.63,78.80,5.82
175,sur,0.0,OM,1.583268e+09,63.0,22.57,59.53,70.03,2.66


In [42]:
hotel_vicinity=[]
hotel_name= []

# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": ['lodging','establisment'],
    'keyword':'hotel',
    "key": g_key
}
# Use the lat/lng we recovered to identify hotels
for index, row in cities_ideal.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place

    params['location'] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng

    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)
    
   # convert to json
    places_data = response.json()
    #print(json.dumps(places_data, indent=4, sort_keys=True))

    try:
        hotel_vicinity.append(places_data['results'][0]['vicinity'])
        hotel_name.append(places_data['results'][0]['name'])
    except:
        print("Data not found... Skipping")
        hotel_vicinity.append(None)
        hotel_name.append(None)
        


Data not found... Skipping
Data not found... Skipping
Data not found... Skipping
Data not found... Skipping
Data not found... Skipping
Data not found... Skipping
Data not found... Skipping


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df['Hotel Name'] = hotel_name

hotel_df = hotel_df.dropna(axis = 0)

hotel_df

C:\Users\sasmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
0,bosaso,0.0,SO,1.583268e+09,77.0,11.28,49.18,76.87,9.48,Gacayte Hotel
53,akyab,0.0,MM,1.583268e+09,86.0,20.15,92.90,71.69,8.84,Hotel Memory
62,hermanus,0.0,ZA,1.583268e+09,68.0,-34.42,19.23,75.00,9.95,The Marine Hermanus
79,cape town,0.0,ZA,1.583268e+09,73.0,-33.93,18.42,75.00,1.12,Atlanticview Cape Town Boutique Hotel
89,bredasdorp,0.0,ZA,1.583268e+09,68.0,-34.53,20.04,73.00,2.64,Firlane House
152,carnarvon,0.0,AU,1.583268e+09,100.0,-24.87,113.63,78.80,5.82,The Gascoyne Hotel
175,sur,0.0,OM,1.583268e+09,63.0,22.57,59.53,70.03,2.66,Sur Plaza Hotel
176,lebu,0.0,CL,1.583268e+09,48.0,-37.62,-73.65,70.47,8.99,Hotel Plaza Lebu
232,umm lajj,0.0,SA,1.583268e+09,24.0,25.02,37.27,72.18,5.73,عروس الشاطئ للوحدات السكنية المفروشة
240,kouroussa,0.0,GN,1.583268e+09,15.0,10.65,-9.88,79.90,2.01,Hotel Paillote


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations1 = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map

locations = cities_df[['Lat','Lng']]
humidity = cities_df["Humidity"]

# Plot Heatmap
fig1 = gmaps.figure(center=(0, 0), zoom_level=2)

# Create heat layer
heat_layer1 = gmaps.heatmap_layer(locations, weights=humidity,opacity=0.5,point_radius=15)
                               
                               


# Add layer
fig1.add_layer(heat_layer1)
marker_layer = gmaps.marker_layer(locations1,info_box_content=hotel_info)

fig1.add_layer(marker_layer)
fig1
# Display Map

Figure(layout=FigureLayout(height='420px'))